# Sensitivity Analysis ML


- 核心目的
进行超参数敏感性分析并生成多个dataframe。记录在不同超参数组合下的：
- 过拟合score
- Accuracy
- Precision
- Recall
- F1
- 最终预测输出的mean probability

- 业务逻辑
1. 设计正交表敏感性分析实验（L18(3^6)正交表，将729次实验减少到18次）
2. 使用模块化的训练函数（已存储在function目录下）
3. 针对不同参数组合执行敏感性分析
4. 输出目录：`Supplymentary/ML_sensitivity`

- 超参数配置
- **基础配置**：learning_rate=0.001, resnet_layers=[128,128,64], d_model=64, num_heads=4, num_layers=2, dropout_rate=0.3
- **敏感性测试参数**：每个参数3个水平，共6个参数
- **不进行敏感性测试**：batch_size, epochs



In [1]:
# ========== 环境配置 ==========
import warnings
import os
import sys

# 1. 设置环境变量
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 2. 直接重定向stderr（临时）
_old_stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

# 3. 导入matplotlib和tensorflow
try:
    import matplotlib
    import matplotlib.font_manager as fm
    matplotlib.rcParams['font.family'] = 'DejaVu Sans'
    
    import tensorflow as tf
    tf.get_logger().setLevel('ERROR')
finally:
    # 4. 恢复stderr
    sys.stderr.close()
    sys.stderr = _old_stderr

# 5. 设置警告过滤
import sklearn
warnings.filterwarnings('ignore', category=sklearn.base.InconsistentVersionWarning)
warnings.filterwarnings('ignore', module='matplotlib')


import os
import time
import tensorflow as tf

print("="*80)
print("GPU环境配置检查 - WSL Ubuntu + bayes-gpu")
print("="*80)

# ---------------------------
# 0. 基本信息（可选）
# ---------------------------
print(f"TensorFlow 版本: {tf.__version__}")
# 尝试打印构建时 CUDA/cuDNN 版本（不同版本字段名可能不同）
try:
    from tensorflow.sysconfig import get_build_info
    bi = get_build_info()
    print("CUDA build:", bi.get("cuda_version"))
    print("cuDNN build:", bi.get("cudnn_version"))
except Exception:
    pass

# ---------------------------
# 1. 检查并配置 GPU
# ---------------------------
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    print(f"\n✅ 检测到 {len(physical_gpus)} 个 GPU:")
    # 先配置显存策略（必须在任何 GPU 初始化之前）
    try:
        for g in physical_gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("✅ GPU 显存动态增长已启用")
    except RuntimeError as e:
        print(f"⚠️ 显存动态增长设置失败（可能已初始化）：{e}")

    # 打印设备细节
    for i, g in enumerate(physical_gpus):
        print(f"   GPU {i}: {g.name}")
        try:
            det = tf.config.experimental.get_device_details(g)
            # 常见键：'device_name', 'compute_capability'
            print(f"       设备详情: {det}")
        except Exception:
            pass
else:
    print("\n⚠️ 未检测到 GPU，将使用 CPU")

# ---------------------------
# 2. 小测试：GPU 计算与计时
# ---------------------------
print("\n" + "="*80)
print("GPU性能测试")
print("="*80)

use_gpu = len(physical_gpus) > 0
device_str = "/GPU:0" if use_gpu else "/CPU:0"

# 暖机（避免第一次调用的加载开销影响计时）
with tf.device(device_str):
    _ = tf.matmul(tf.random.normal([256, 256]), tf.random.normal([256, 256]))  # warm-up
    _ = tf.matmul(tf.random.normal([256, 256]), tf.random.normal([256, 256]))  # warm-up

# 计时测试（确保同步）
size = 2000  # 如需更明显差异可调大到 4096
with tf.device(device_str):
    a = tf.random.normal([size, size])
    b = tf.random.normal([size, size])
    t0 = time.time()
    c = tf.matmul(a, b)
    _ = c.numpy()  # 同步到主机，确保计时真实
    elapsed_ms = (time.time() - t0) * 1000

print(f"✅ 设备: {device_str}")
print(f"✅ 矩阵乘法 {size}x{size} 耗时: {elapsed_ms:.2f} ms")
if use_gpu:
    print("🚀 GPU 加速正常（与 CPU 相比应明显更快）")
print("="*80)


GPU环境配置检查 - WSL Ubuntu + bayes-gpu
TensorFlow 版本: 2.19.0
CUDA build: None
cuDNN build: None

⚠️ 未检测到 GPU，将使用 CPU

GPU性能测试
✅ 设备: /CPU:0
✅ 矩阵乘法 2000x2000 耗时: 39.61 ms


## 1、Load data

In [ ]:
import sys
import os
from pathlib import Path

# 最可靠的方法：查找包含data和function目录的项目根目录
def find_project_root(start_path=None):
    """查找项目根目录（包含data和function目录的目录）"""
    if start_path is None:
        start_path = Path.cwd()
    
    current = Path(start_path).resolve()
    
    # 向上查找，直到找到包含data和function目录的目录
    for _ in range(5):  # 最多向上查找5层
        if (current / 'data').exists() and (current / 'function').exists():
            return current
        parent = current.parent
        if parent == current:  # 到达根目录
            break
        current = parent
    
    # 如果找不到，假设当前目录的父目录是项目根目录
    return Path.cwd().parent

project_root = find_project_root()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

DATA_PATH = project_root / 'data'

print(f"项目根目录: {project_root}")
print(f"数据路径: {DATA_PATH}")


项目根目录: /mnt/c/Dev/Landuse_Zhong_clean
数据路径: /mnt/c/Dev/Landuse_Zhong_clean/data


In [2]:
%load_ext autoreload
%autoreload 2 
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import geopandas as gpd
from shapely.geometry import Point
from function import (
    load_embedding, load_abandon, fill_nonpositive_with_nearest, 
    filter_duplicates, NUMERIC_FEATURES, CAT_COLS, PATHS
)
from function.pipeline import run_correct_training_pipeline
from function.model_saving import save_complete_model_pipeline
from datetime import datetime
import platform
from tqdm.auto import tqdm

def normalize_path(path):
    """
    跨平台路径规范化
    将Windows路径中的反斜杠转换为正斜杠（在Linux/WSL中）
    Windows: r'data\file.shp' -> r'data\file.shp' (保持不变)
    Linux/WSL: r'data\file.shp' -> 'data/file.shp' (转换)
    """
    if isinstance(path, str):
        # 检测是否在WSL或Linux环境中
        is_linux = platform.system() in ['Linux', 'Darwin']
        
        if is_linux:
            # 在Linux/WSL中，将反斜杠转为正斜杠
            path = path.replace('\\', '/')
        
        return path
    return path

def normalize_paths_dict(paths_dict):
    """
    规范化路径字典中的所有路径
    """
    normalized = {}
    for key, value in paths_dict.items():
        normalized[key] = normalize_path(value)
    return normalized

def clip_data_with_us_states(df, us_states_gdf, lon_col='lon', lat_col='lat'):
    """
    使用美国州界 shapefile 剪裁点数据；兼容不同版本的 geopandas 参数名
    """
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    us_states_4326 = us_states_gdf.to_crs('EPSG:4326')

    try:
        clipped = gpd.sjoin(gdf, us_states_4326, how='inner', predicate='within')
    except TypeError:
        # 旧版本 geopandas 使用 op 参数
        clipped = gpd.sjoin(gdf, us_states_4326, how='inner', op='within')

    # 清理 shapefile 附加字段
    clipped = clipped.drop(columns=['geometry', 'index_right'], errors='ignore')
    for col in us_states_gdf.columns:
        if col in clipped.columns:
            clipped = clipped.drop(columns=[col], errors='ignore')
    return clipped

PATHS = normalize_paths_dict(PATHS)
PATHS['us_pv_embedding'] = normalize_path(str(DATA_PATH / 'training_embedding.csv'))
PATHS['us_abandon'] = normalize_path(str(DATA_PATH / 'us_abandon_clean.csv'))
print(f"us_abandon path: {PATHS['us_abandon']}")


usa_bounds_main = dict(lon_min=-125, lon_max=-65, lat_min=25, lat_max=49)
# 加载数据
print("加载数据...")
df_embedding = load_embedding(PATHS['us_pv_embedding'])
df_abandon = load_abandon(PATHS['us_abandon'])

df_abandon = load_abandon(PATHS['us_abandon'])
df_embedding = load_embedding(PATHS['us_pv_embedding'])

# 初步经纬度范围过滤
df_embedding = df_embedding[
    (df_embedding['lon'] >= usa_bounds_main['lon_min']) &
    (df_embedding['lon'] <= usa_bounds_main['lon_max']) &
    (df_embedding['lat'] >= usa_bounds_main['lat_min']) &
    (df_embedding['lat'] <= usa_bounds_main['lat_max'])
]

df_abandon = df_abandon[
    (df_abandon['lon'] >= usa_bounds_main['lon_min']) &
    (df_abandon['lon'] <= usa_bounds_main['lon_max']) &
    (df_abandon['lat'] >= usa_bounds_main['lat_min']) &
    (df_abandon['lat'] <= usa_bounds_main['lat_max'])
]
# 第二次用州界矢量更精确裁剪
us_nation = gpd.read_file(normalize_path(str(DATA_PATH / 'US_data' / 'cb_2018_us_nation_5m.shp')))
df_abandon = clip_data_with_us_states(df_abandon, us_nation)
df_embedding = clip_data_with_us_states(df_embedding, us_nation)

# 数据预处理
print("预处理数据...")
df_embedding_fill = fill_nonpositive_with_nearest(df_embedding)
df_abandon_fill = fill_nonpositive_with_nearest(df_abandon)
df_abandon_filtered = filter_duplicates(df_abandon_fill, df_embedding_fill)

# 定义特征列表（排除坐标）
features_no_coords = [f for f in (NUMERIC_FEATURES + CAT_COLS) if f not in ['lat', 'lon']]
features_no_coords = [c for c in features_no_coords if c in df_embedding_fill.columns]

print(f"✅ 数据加载完成")
print(f"  - df_embedding_fill: {len(df_embedding_fill)} 行")
print(f"  - df_abandon_filtered: {len(df_abandon_filtered)} 行")
print(f"  - features_no_coords: {len(features_no_coords)} 个特征")


gogogo


I0000 00:00:1764019411.733590  147604 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1764019411.878892  147604 cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI AVX_VNNI_INT8 AVX_NE_CONVERT FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1764019413.225357  147604 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[OK] TensorFlow available
[OK] scikeras available
[OK] SHAP available
us_abandon path: /mnt/c/Dev/Landuse_Zhong_clean/data/us_abandon_clean.csv
加载数据...
You want to predict the year: 2020.0
You want to predict the year: 2020.0
预处理数据...
列 GDPpc 没有需要填充的非正值或NaN值
列 GDPpc 没有需要填充的非正值或NaN值
✅ 数据加载完成
  - df_embedding_fill: 10473 行
  - df_abandon_filtered: 70337 行
  - features_no_coords: 15 个特征


## 2、Hyperparameter settings


In [4]:
# 基础配置（base model = 各因子的中等水平）
base_configs = {
    "learning_rate": 0.001,           # 中等
    "dropout_rate": 0.3,              # 中等
    "transformer_config": {
        "d_model": 64,                # 中等宽度
        "num_heads": 4,
        "num_layers": 8,              # ← 中等深度（浅/中/深：4 / 8 / 12）
    },
    "resnet_width": 128,
    "resnet_depth": 6,
}

# 敏感性测试参数（6个因子，每个3个水平）
sensitivity_configs = {
    # 1. Transformer宽度（d_model）
    "d_model": [32, 64, 128],             # 小 / 中 / 大

    # 2. Transformer深度（num_layers）
    "num_layers": [4, 8, 12],             # 浅 / 中 / 深（差异明显）

    # 3. ResNet宽度（第一层宽度）
    "resnet_width": [64, 128, 256],       # 小 / 中 / 大

    # 4. ResNet深度（层数）
    "resnet_depth": [3, 6, 9],            # 浅 / 中 / 深（明显区分）

    # 5. 学习率
    "learning_rate": [0.0001, 0.001, 0.01],  # 小 / 中 / 大

    # 6. Dropout率
    "dropout_rate": [0.1, 0.3, 0.5],      # 弱 / 中 / 强正则
}

# 辅助函数：将resnet_width和resnet_depth转换为resnet_layers列表
def generate_resnet_layers(width, depth):
    layers = []
    current_width = width
    for i in range(depth):
        layers.append(int(current_width))
        if i < depth - 1:  # 最后一层不继续减半
            current_width = current_width / 2
    return layers

# base_model 的 resnet_layers 示例
base_resnet_layers = generate_resnet_layers(128, 6)
# e.g. [128, 64, 32, 16, 8, 4]




|   Exp |   d_model |   num_layers |   resnet_width |   resnet_depth |   learning_rate |   dropout_rate |
|------:|----------:|-------------:|---------------:|---------------:|----------------:|---------------:|
|     1 |        32 |            4 |             64 |              3 |          0.0001 |            0.1 |
|     2 |        32 |            8 |            128 |              6 |          0.001  |            0.3 |
|     3 |        32 |           12 |            256 |              9 |          0.01   |            0.5 |
|     4 |        64 |            4 |             64 |              6 |          0.001  |            0.5 |
|     5 |        64 |            8 |            128 |              9 |          0.01   |            0.1 |
|     6 |        64 |           12 |            256 |              3 |          0.0001 |            0.3 |
|     7 |       128 |            4 |            128 |              3 |          0.01   |            0.3 |
|     8 |       128 |            8 |            256 |              6 |          0.0001 |            0.5 |
|     9 |       128 |           12 |             64 |              9 |          0.001  |            0.1 |
|    10 |        32 |            4 |            256 |              9 |          0.001  |            0.3 |
|    11 |        32 |            8 |             64 |              3 |          0.01   |            0.5 |
|    12 |        32 |           12 |            128 |              6 |          0.0001 |            0.1 |
|    13 |        64 |            4 |            128 |              9 |          0.0001 |            0.5 |
|    14 |        64 |            8 |            256 |              3 |          0.001  |            0.1 |
|    15 |        64 |           12 |             64 |              6 |          0.01   |            0.3 |
|    16 |       128 |            4 |            256 |              6 |          0.001  |            0.1 |
|    17 |       128 |            8 |             64 |              9 |          0.01   |            0.3 |
|    18 |       128 |           12 |            128 |              3 |          0.0001 |            0.5 |


In [5]:
# 3. 正交实验设计（L18(3^6)）
def generate_l18_orthogonal_array():
    """
    生成L18(3^6)标准正交表
    18行（实验次数）× 6列（因子数）
    每列包含0,1,2三个水平的平衡分布
    """
    # 标准L18(3^6)正交表
    orthogonal_array = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 2, 2, 2, 2, 2],
        [1, 0, 0, 1, 1, 2],
        [1, 1, 1, 2, 2, 0],
        [1, 2, 2, 0, 0, 1],
        [2, 0, 1, 0, 2, 1],
        [2, 1, 2, 1, 0, 2],
        [2, 2, 0, 2, 1, 0],
        [0, 0, 2, 2, 1, 1],
        [0, 1, 0, 0, 2, 2],
        [0, 2, 1, 1, 0, 0],
        [1, 0, 1, 2, 0, 2],
        [1, 1, 2, 0, 1, 0],
        [1, 2, 0, 1, 2, 1],
        [2, 0, 2, 1, 1, 0],
        [2, 1, 0, 2, 2, 1],
        [2, 2, 1, 0, 0, 2]
    ])
    return orthogonal_array

# 生成正交表
orthogonal_array = generate_l18_orthogonal_array()
print("✅ L18(3^6)正交表生成完成")
print(f"  - 实验次数: {len(orthogonal_array)} (全因子设计需要 {3**6} 次)")
print(f"  - 减少实验次数: {3**6 - len(orthogonal_array)} 次 ({100*(1-len(orthogonal_array)/(3**6)):.1f}%)")
print(f"  - 正交表形状: {orthogonal_array.shape}")


✅ L18(3^6)正交表生成完成
  - 实验次数: 18 (全因子设计需要 729 次)
  - 减少实验次数: 711 次 (97.5%)
  - 正交表形状: (18, 6)


In [6]:
# 4. 参数映射函数（更新版：处理ResNet宽度和深度）
def map_orthogonal_to_params(orthogonal_array, param_levels):
    """
    将正交表中的水平索引映射到实际参数值
    特殊处理：将resnet_width和resnet_depth转换为resnet_layers列表
    
    参数:
    - orthogonal_array: 18×6的正交表数组
    - param_levels: 参数字典，每个参数对应3个水平值的列表
    
    返回:
    - param_combinations: 参数组合列表（包含转换后的resnet_layers）
    """
    param_names = list(param_levels.keys())
    param_combinations = []
    
    for row in orthogonal_array:
        param_dict = {}
        for i, param_name in enumerate(param_names):
            level_idx = row[i]
            param_dict[param_name] = param_levels[param_name][level_idx]
        
        # 特殊处理：将resnet_width和resnet_depth转换为resnet_layers
        if 'resnet_width' in param_dict and 'resnet_depth' in param_dict:
            resnet_layers = generate_resnet_layers(
                param_dict['resnet_width'], 
                param_dict['resnet_depth']
            )
            param_dict['resnet_layers'] = resnet_layers
            # 保留原始值用于记录
            param_dict['_resnet_width'] = param_dict.pop('resnet_width')
            param_dict['_resnet_depth'] = param_dict.pop('resnet_depth')
        
        # 确保num_heads与d_model兼容（d_model必须能被num_heads整除）
        if 'd_model' in param_dict:
            d_model = param_dict['d_model']
            # 根据d_model自动选择合适的num_heads
            if d_model == 32:
                param_dict['num_heads'] = 2  # 32/2=16
            elif d_model == 64:
                param_dict['num_heads'] = 4  # 64/4=16
            elif d_model == 96:
                param_dict['num_heads'] = 4  # 96/4=24（或8，但4更稳定）
        
        param_combinations.append(param_dict)
    
    return param_combinations

# 生成参数组合
param_combinations = map_orthogonal_to_params(orthogonal_array, sensitivity_configs)
print("✅ 参数组合映射完成")
print(f"  - 共 {len(param_combinations)} 个参数组合")
print(f"  - 示例组合 0:")
for k, v in param_combinations[0].items():
    print(f"      {k}: {v}")


✅ 参数组合映射完成
  - 共 18 个参数组合
  - 示例组合 0:
      d_model: 32
      num_layers: 4
      learning_rate: 0.0001
      dropout_rate: 0.1
      resnet_layers: [64, 32, 16]
      _resnet_width: 64
      _resnet_depth: 3
      num_heads: 2



敏感性分析完成后，将生成以下6个DataFrame，每个DataFrame包含18个实验的参数组合和对应的指标值：

| DataFrame名称 | 指标列 | 说明 | 保存文件名 |
|--------------|--------|------|-----------|
| `df_overfitting` | `overfitting_score` | 过拟合分数（训练集与验证集的最终差异） | `df_overfitting.csv` |
| `df_accuracy` | `accuracy` | 测试集准确率 | `df_accuracy.csv` |
| `df_precision` | `precision` | 测试集精确率 | `df_precision.csv` |
| `df_recall` | `recall` | 测试集召回率 | `df_recall.csv` |
| `df_f1` | `f1` | 测试集F1分数 | `df_f1.csv` |
| `df_mean_prob` | `mean_probability` | 预测结果的平均概率 | `df_mean_prob.csv` |



每个DataFrame包含以下列：

**参数列（6列）**：
- `learning_rate`: 学习率
- `resnet_layers_str`: ResNet层结构（字符串格式，如 "[128, 128, 64]"）
- `d_model`: Transformer模型维度
- `num_heads`: Transformer注意力头数
- `num_layers`: Transformer层数
- `dropout_rate`: Dropout比率

**指标列（1列）**：
- 对应的指标值（根据DataFrame类型不同）


所有结果将保存到 `Supplymentary/ML_sensitivity/` 目录：
- 6个CSV文件（每个DataFrame一个）
- `orthogonal_config.json`：正交表配置信息
- `sensitivity_summary.json`：结果摘要（包括最佳参数组合）



## 3、Sensitivity analysis

In [7]:
# 5. 敏感性分析主函数
from typing import Any


def extract_metrics(result, params):
    """
    从训练结果中提取指标
    
    参数:
    - result: run_correct_training_pipeline的返回结果
    - params: 参数字典
    
    返回:
    - metrics_dict: 包含所有指标的字典
    """
    metrics_dict = params.copy()
    
    try:
        # 过拟合score
        if result.get('learning_curve_analysis') and result['learning_curve_analysis'].get('overfitting_analysis'):
            metrics_dict['overfitting_score'] = result['learning_curve_analysis']['overfitting_analysis'].get('final_gap', np.nan)
        else:
            metrics_dict['overfitting_score'] = np.nan
        
        # 测试集指标
        if result.get('training_results') and result['training_results'].get('metrics'):
            test_metrics = result['training_results']['metrics'].get('test', {})
            metrics_dict['accuracy'] = test_metrics.get('accuracy', np.nan)
            metrics_dict['precision'] = test_metrics.get('precision', np.nan)
            metrics_dict['recall'] = test_metrics.get('recall', np.nan)
            metrics_dict['f1'] = test_metrics.get('f1', np.nan)
        else:
            metrics_dict['accuracy'] = np.nan
            metrics_dict['precision'] = np.nan
            metrics_dict['recall'] = np.nan
            metrics_dict['f1'] = np.nan
        
        # mean probability
        if result.get('prediction_results') is not None:
            pred_df = result['prediction_results']
            if 'predicted_prob' in pred_df.columns:
                metrics_dict['mean_probability'] = pred_df['predicted_prob'].mean()
            else:
                metrics_dict['mean_probability'] = np.nan
        else:
            metrics_dict['mean_probability'] = np.nan
            
    except Exception as e:
        print(f"⚠️ 提取指标时出错: {e}")
        metrics_dict['overfitting_score'] = np.nan
        metrics_dict['accuracy'] = np.nan
        metrics_dict['precision'] = np.nan
        metrics_dict['recall'] = np.nan
        metrics_dict['f1'] = np.nan
        metrics_dict['mean_probability'] = np.nan
    
    return metrics_dict

def run_sensitivity_analysis(
    df_positive, df_prediction_pool, features_no_coords,
    param_combinations, base_configs,
    epochs=80, batch_size=256, random_state=42,
    test_size=0.2, val_size=0.2,
    negative_strategy='generation', negative_ratio=1,
    plot_learning_curve=True, learning_curve_epochs=20,
    output_dir='Supplymentary/ML_sensitivity',
    save_models=True
):
    """
    运行敏感性分析
    
    参数:
    - df_positive: 正样本数据
    - df_prediction_pool: 预测池数据
    - features_no_coords: 特征列表
    - param_combinations: 参数组合列表
    - base_configs: 基础配置
    - 其他参数: 训练相关参数
    
    返回:
    - results: 所有实验的结果列表
    """
    results = []
    saved_models = []  # 记录保存的模型路径
    
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    models_dir = os.path.join(output_dir, 'models')
    os.makedirs(models_dir, exist_ok=True)
    
    print(f"\n{'='*80}")
    print(f"开始敏感性分析: {len(param_combinations)} 个实验")
    print(f"负样本策略: {negative_strategy}")
    print(f"模型保存目录: {models_dir}")
    print(f"{'='*80}\n")
    
    # 使用tqdm显示总体进度，并配置适合WSL/Jupyter的显示方式
    pbar = tqdm(enumerate(param_combinations), 
                total=len(param_combinations),
                desc="敏感性分析",
                unit="实验",
                ncols=100,  
                mininterval=1.0,  
                bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')
    
    for i, params in pbar:
        exp_id = f"E{i+1}"
        # 更新进度条描述，显示当前实验信息
        pbar.set_description(f"实验 {exp_id}")
        pbar.set_postfix({
            'LR': f"{params['learning_rate']:.4f}",
            'd_m': params['d_model']
        })
        
        exp_id = f"E{i+1}"
        print(f"\n实验 {exp_id} ({i+1}/{len(param_combinations)})")
        print(f"参数组合: {params}")
        
        try:
            # 构建transformer_config（使用更新后的参数结构）
            transformer_config = {
                'd_model': params['d_model'],
                'num_heads': params.get('num_heads', 4),  # 从映射函数中已设置
                'num_layers': params['num_layers']
            }
            
            # 获取ResNet层配置（已从width和depth转换）
            resnet_layers = params['resnet_layers']
            
            # 调用训练管道
            result = run_correct_training_pipeline(
                df_positive=df_positive,
                df_prediction_pool=df_prediction_pool,
                features_no_coords=features_no_coords,
                negative_strategy=negative_strategy,
                negative_ratio=negative_ratio,
                augmentation_ratio=1,  
                test_size=test_size,
                val_size=val_size,
                epochs=epochs,
                batch_size=batch_size,
                random_state=random_state,
                learning_rate=params['learning_rate'],
                dropout_rate=params['dropout_rate'],
                resnet_layers=resnet_layers,
                transformer_config=transformer_config,
                model_type='transformer',
                train_mode='single',
                plot_learning_curve=plot_learning_curve,
                learning_curve_epochs=learning_curve_epochs,
                run_shap=False
            )
            
            # 提取指标
            metrics = extract_metrics(result, params)
            results.append(metrics)
            
            if save_models and result is not None:
                try:
                    model_name = f"{exp_id}_transformer_generation"
                    saved_path = save_complete_model_pipeline(
                        gmm_pipeline=result.get('gmm_pipeline'),
                        dl_model=result.get('model'),
                        retrained_preprocessor=result['training_results'].get('preprocessor'),
                        training_results=result.get('training_results'),
                        final_results=result.get('final_results'),
                        negative_results=result.get('negative_samples'),
                        prediction_results=result.get('prediction_results'),
                        features=features_no_coords,
                        config=result.get('config', {}),
                        save_dir=models_dir,
                        model_name=model_name,
                        model_type='transformer',
                        negative_strategy=negative_strategy,
                        train_mode='single',
                        pu_evaluation=result.get('pu_evaluation')
                    )
                    saved_models.append({
                        'exp_id': exp_id,
                        'exp_num': i+1,
                        'model_path': saved_path,
                        'params': params,
                        'metrics': {k: v for k, v in metrics.items() 
                                   if k in ['accuracy', 'f1', 'overfitting_score', 'precision', 'recall', 'mean_probability']}
                    })
                    print(f"  💾 模型已保存: {saved_path}")
                except Exception as save_error:
                    print(f"  ⚠️ 模型保存失败: {save_error}")
            
            # 更新进度条后置信息
            pbar.set_postfix({
                'Acc': f"{metrics.get('accuracy', np.nan):.3f}",
                'F1': f"{metrics.get('f1', np.nan):.3f}"
            })
            
            print(f"✅ 实验 {exp_id} 完成")
            print(f"  - Accuracy: {metrics.get('accuracy', 'N/A'):.4f}")
            print(f"  - F1: {metrics.get('f1', 'N/A'):.4f}")
            print(f"  - 过拟合score: {metrics.get('overfitting_score', 'N/A'):.4f}")
            
        except Exception as e:
            print(f"❌ 实验 {exp_id} 失败: {e}")
            import traceback
            traceback.print_exc()
            # 记录失败实验的参数
            failed_metrics = params.copy()
            failed_metrics['overfitting_score'] = np.nan
            failed_metrics['accuracy'] = np.nan
            failed_metrics['precision'] = np.nan
            failed_metrics['recall'] = np.nan
            failed_metrics['f1'] = np.nan
            failed_metrics['mean_probability'] = np.nan
            results.append(failed_metrics)
    
    
    # 关闭进度条
    pbar.close()
    
    # 保存模型索引文件
    if save_models and saved_models:
        models_index_file = os.path.join(output_dir, 'models_index.json')
        with open(models_index_file, 'w', encoding='utf-8') as f:
            json.dump(saved_models, f, indent=2, ensure_ascii=False, default=str)
        print(f"\n✅ 模型索引已保存: {models_index_file}")
    
    print(f"\n{'='*80}")
    print(f"敏感性分析完成: {len(results)}/{len(param_combinations)} 个实验成功")
    if save_models:
        print(f"已保存模型: {len(saved_models)}/{len(param_combinations)} 个")
    print(f"{'='*80}\n")
    
    return results

print("✅ 敏感性分析函数定义完成")


✅ 敏感性分析函数定义完成


In [ ]:
results = run_sensitivity_analysis(
    df_positive=df_embedding_fill,  
    df_prediction_pool=df_abandon_filtered,  
    features_no_coords=features_no_coords,
    param_combinations=param_combinations,
    base_configs=base_configs,
    epochs=80,
    batch_size=256,  
    random_state=42,
    test_size=0.2,
    val_size=0.2,
    negative_strategy='generation',  
    negative_ratio=1,  
    plot_learning_curve=True,
    learning_curve_epochs=20,
    output_dir='Supplymentary/ML_sensitivity', 
    save_models=True  
)


开始敏感性分析: 18 个实验
负样本策略: generation
模型保存目录: Supplymentary/ML_sensitivity/models



敏感性分析:   0%|                                                          | 0/18 [00:00<?, ?实验/s]


实验 E1 (1/18)
参数组合: {'d_model': 32, 'num_layers': 4, 'learning_rate': 0.0001, 'dropout_rate': 0.1, 'resnet_layers': [64, 32, 16], '_resnet_width': 64, '_resnet_depth': 3, 'num_heads': 2}
正确的训练管道：分层负样本采样的完整流程

步骤1: 加载或训练GMM模型用于环境相似度评估
[GMM] 当前项目根目录推断为: /mnt/c/Dev/Landuse_Zhong_clean
🔍 发现已保存的GMM模型文件: /mnt/c/Dev/Landuse_Zhong_clean/gmm_model_23c_fixed.pkl
📂 尝试加载模型...
✅ 成功加载GMM模型: gmm_model_23c_fixed.pkl
   模型组件数: 23
   协方差类型: diag

步骤2: 负样本生成 - 策略: generation

统一负样本生成接口 - 策略: generation

生成式负样本策略（纯粹GMM采样）

生成配置:
  正样本数: 10473
  目标负样本数: 10473

筛选标准:
  正样本平均log概率: 49.978
  阈值 (均值 - 1std): 27.508

尝试 1/8: 采样 20946 个...
  实际采样: 20946 个
  符合阈值: 217 个 (1.0%)
  累计有效样本: 217 个
  ⏳ 继续增加采样量 (下次: 31419 个)...

尝试 2/8: 采样 31419 个...


/home/zpy10/bayes-gpu/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.7.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/zpy10/bayes-gpu/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/zpy10/bayes-gpu/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.7.0 when using version 1.7.2. This might

  实际采样: 31419 个
  符合阈值: 359 个 (1.1%)
  累计有效样本: 576 个
  ⏳ 继续增加采样量 (下次: 41892 个)...

尝试 3/8: 采样 41892 个...
  实际采样: 41892 个
  符合阈值: 476 个 (1.1%)
  累计有效样本: 1052 个
  ⏳ 继续增加采样量 (下次: 62838 个)...

尝试 4/8: 采样 62838 个...
  实际采样: 62838 个
  符合阈值: 710 个 (1.1%)
  累计有效样本: 1762 个
  ⏳ 继续增加采样量 (下次: 94257 个)...

尝试 5/8: 采样 94257 个...
  实际采样: 94257 个
  符合阈值: 1070 个 (1.1%)
  累计有效样本: 2832 个
  ⏳ 继续增加采样量 (下次: 136149 个)...

尝试 6/8: 采样 136149 个...
  实际采样: 136149 个
  符合阈值: 1497 个 (1.1%)
  累计有效样本: 4329 个
  ⏳ 继续增加采样量 (下次: 198987 个)...

尝试 7/8: 采样 198987 个...
  实际采样: 198987 个
  符合阈值: 2174 个 (1.1%)
  累计有效样本: 6503 个
  ⏳ 继续增加采样量 (下次: 293244 个)...

尝试 8/8: 采样 293244 个...
  实际采样: 293244 个
  符合阈值: 3151 个 (1.1%)
  累计有效样本: 9654 个
  ⚠️ 已达到最大尝试次数，使用所有可用样本 (9654 个)

采样统计:
  总采样次数: 879732 个
  有效样本率: 1.10%
  最终负样本数: 9654 个

逆变换映射到原始特征空间（批量处理）...
逆变换完成: 9654 个负样本
平均log-density: -52.238
生成完成: 9654 个负样本
平均log-density: -52.238
保留整个预测池用于预测: 70337 个样本

步骤3: 训练深度学习分类模型
✅ TensorFlow确定性模式已启用
准备原始数据...
原始特征: (20127, 15)
标签分布: 正样本=10473, 

...

E0000 00:00:1764019848.166257  147604 node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


    Fold 2/5 | train_score=0.837, val_score=0.820
Building Transformer+ResNet Hybrid Model...
  Input dim: 23 | Transformer layers: 4 | Params: 61,837


E0000 00:00:1764019964.835992  147604 node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


    Fold 3/5 | train_score=0.822, val_score=0.806
Building Transformer+ResNet Hybrid Model...
  Input dim: 23 | Transformer layers: 4 | Params: 61,837


KeyboardInterrupt: 

In [ ]:
# 7. 结果整理为DataFrame
def create_dataframes(results):
    """
    将结果列表转换为多个DataFrame
    
    参数:
    - results: 结果列表，每个元素是一个包含参数和指标的字典
    
    返回:
    - dataframes: 包含6个DataFrame的字典
    """
    # 准备数据（使用更新后的参数结构）
    data = []
    for r in results:
        row = {
            'd_model': r.get('d_model', np.nan),  # Transformer宽度
            'num_layers': r.get('num_layers', np.nan),  # Transformer深度
            'resnet_width': r.get('_resnet_width', np.nan),  # ResNet宽度（第一层）
            'resnet_depth': r.get('_resnet_depth', np.nan),  # ResNet深度（层数）
            'resnet_layers_str': str(r.get('resnet_layers', [])),  # 完整ResNet配置（字符串）
            'learning_rate': r.get('learning_rate', np.nan),
            'dropout_rate': r.get('dropout_rate', np.nan),
            'num_heads': r.get('num_heads', np.nan),  # 自动计算的注意力头数
            'overfitting_score': r.get('overfitting_score', np.nan),
            'accuracy': r.get('accuracy', np.nan),
            'precision': r.get('precision', np.nan),
            'recall': r.get('recall', np.nan),
            'f1': r.get('f1', np.nan),
            'mean_probability': r.get('mean_probability', np.nan)
        }
        data.append(row)
    
    df_all = pd.DataFrame(data)
    
    # 创建各个指标的DataFrame
    # 参数列：宽度、深度、学习率、dropout
    param_cols = [
        'd_model',           # Transformer宽度
        'num_layers',        # Transformer深度
        'resnet_width',      # ResNet宽度
        'resnet_depth',      # ResNet深度
        'learning_rate',     # 学习率
        'dropout_rate'       # Dropout率
    ]
    
    dataframes = {
        'df_overfitting': df_all[param_cols + ['overfitting_score']].copy(),
        'df_accuracy': df_all[param_cols + ['accuracy']].copy(),
        'df_precision': df_all[param_cols + ['precision']].copy(),
        'df_recall': df_all[param_cols + ['recall']].copy(),
        'df_f1': df_all[param_cols + ['f1']].copy(),
        'df_mean_prob': df_all[param_cols + ['mean_probability']].copy()
    }
    
    return dataframes

# 创建DataFrame
dataframes = create_dataframes(results)

print("✅ DataFrame创建完成")
for name, df in dataframes.items():
    print(f"  - {name}: {df.shape}")
    print(f"    缺失值: {df.isna().sum().sum()}")


In [ ]:
# 8. 保存结果
output_dir = 'Supplymentary/ML_sensitivity'
os.makedirs(output_dir, exist_ok=True)

# 保存各个DataFrame
for name, df in dataframes.items():
    filepath = os.path.join(output_dir, f"{name}.csv")
    df.to_csv(filepath, index=False)
    print(f"✅ 已保存: {filepath}")

# 保存正交表配置信息
config_info = {
    'orthogonal_array': orthogonal_array.tolist(),
    'sensitivity_configs': {k: [str(v) if isinstance(v, list) else v for v in vals] 
                            for k, vals in sensitivity_configs.items()},
    'base_configs': {k: (str(v) if isinstance(v, list) else v) 
                     for k, v in base_configs.items()},
    'num_experiments': len(param_combinations),
    'full_factorial_experiments': 3**6
}

config_filepath = os.path.join(output_dir, 'orthogonal_config.json')
with open(config_filepath, 'w', encoding='utf-8') as f:
    json.dump(config_info, f, indent=2, ensure_ascii=False)
print(f"✅ 已保存配置信息: {config_filepath}")

# 保存完整结果摘要
summary = {
    'total_experiments': len(results),
    'successful_experiments': sum(1 for r in results if not np.isnan(r.get('accuracy', np.nan))),
    'failed_experiments': sum(1 for r in results if np.isnan(r.get('accuracy', np.nan))),
    'best_accuracy': max([r.get('accuracy', -np.inf) for r in results], default=np.nan),
    'best_f1': max([r.get('f1', -np.inf) for r in results], default=np.nan),
    'best_params_accuracy': None,
    'best_params_f1': None
}

# 找到最佳参数组合
if not np.isnan(summary['best_accuracy']):
    best_acc_idx = np.argmax([r.get('accuracy', -np.inf) for r in results])
    summary['best_params_accuracy'] = {k: v for k, v in results[best_acc_idx].items() 
                                       if k not in ['overfitting_score', 'accuracy', 'precision', 'recall', 'f1', 'mean_probability']}

if not np.isnan(summary['best_f1']):
    best_f1_idx = np.argmax([r.get('f1', -np.inf) for r in results])
    summary['best_params_f1'] = {k: v for k, v in results[best_f1_idx].items() 
                                 if k not in ['overfitting_score', 'accuracy', 'precision', 'recall', 'f1', 'mean_probability']}

summary_filepath = os.path.join(output_dir, 'sensitivity_summary.json')
with open(summary_filepath, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False, default=str)
print(f"✅ 已保存结果摘要: {summary_filepath}")

print(f"\n{'='*80}")
print(f"所有结果已保存到: {output_dir}")
print(f"{'='*80}\n")
